<a href="https://colab.research.google.com/github/akhilesh-somani/Restaurant_Recommendation_System/blob/main/Baseline1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np



In [18]:
df=pd.read_csv("champaign_user_item.csv")
df.columns

Index(['Unnamed: 0', 'business_id', 'name', 'address', 'city', 'state',
       'postal_code', 'latitude', 'longitude', 'stars_x', 'review_count',
       'is_open', 'attributes', 'categories', 'hours', 'review_id', 'user_id',
       'stars_y', 'useful', 'funny', 'cool', 'text', 'date'],
      dtype='object')

In [19]:
df_review = df[['user_id','business_id','stars_y', 'date', 'name', 'address']]


In [20]:
y=df_review.stars_y
X=df[['user_id','business_id', 'date', 'name', 'address']]

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [22]:
rest_test_data=pd.concat([X_train, pd.DataFrame(y_train)], ignore_index=True, axis=1)
rest_test_data.columns=['user_id','business_id', 'date', 'name', 'address', 'stars_y']


In [23]:
avg_rating=rest_test_data.groupby('name')['stars_y'].mean().sort_values(ascending=False)

In [24]:
#avg_rating.loc['U-Haul Moving & Storage at Southgate']

In [25]:
rating_crosstab = rest_test_data.pivot_table(values='stars_y', index='user_id', columns='name', fill_value=0)
rating_crosstab.head()

name,#1 Nails,1 Man 2 Hands,217,217 & Smith's Towing,217 Immediate Care,2M Cleaning Services,301 Mongolia,309 Green,4U Nails & Spa,5 Star Nutrition Champaign,88 West Apartments,8F Ice Cream & Crepe,901 Western,A & E Animal Hospital,A & K Mega Service,A Hunt Design,A Plus VIP Lounge,A To Z Appliance Repair,A&B Scissor Hands,A-Ri-Rang,A1 Arch Salon,A1 Cleaning Service,ABC Heating & Air Conditioning,ABC Sanitary,AMC Champaign 13,ASPCA Animal Poison Control Center,AT&T Store,ATLAS Cab,Abbott's Florist,Above and Beyond Detailing,Ace Hardware,Activities and Recreation Center - ARC,Acupuncture & Chinese Herb Center,Advance Auto Parts,Advance Muffler Plus,Advanced Aesthetics Surgery & Laser Center,Advantage Hobby,Agape Hearts And Hand Alternative Therapies,Albert Jodi & Associates,Aldi,...,White Horse Inn,Wicked Rascal Barbershop,Wienerschnitzel,Wilbur's Taxi,Wild Side Smoke Shop,Will Am-Fm Radio,William M Staerkel Planetarium,Window Genie of Champaign,Windsor West Apartments,Windy City Express,Wines At the Pines,Wingard Salon,Wingate By Wyndham Champaign,Wingin Out,Wingstop,"Wireless Mikes, Verizon Authorized Retailer",Witte Electric,Women's Health Practice,Wonderdogs,Wood N' Hog Barbecue,Wood Nâ Hog Barbeque,Woori Jib Restaurant,Worden-Martin Buick GMC,World Harvest Foods,World Nail Spa,Wyndham Garden Urbana Champaign,Xinh Xinh Cafe,YMCA,Ye Olde Donut Shoppe,Yellow Checker Cab,Yellowfin Japanese Restaurant,Yoga Institute of Champaign-Urbana,Yong's Alterations,Yori Q Korean Grill & Bar,Yuko Hair Salon,Za's Italian Cafe,Zelma's,Zmodo Technology,Zorba's Restaurant,Zyggyz
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
--ERQVpqAAoi262TTbLVzQ,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0
-06IbFmaohdrg7kQKc3A4A,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0
-0e6xyw_4zyg-2YtqSlS_g,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0,0,0,0,0,0,0,0,3,0.0,0,0,0,0,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0
-0nK53RBD052LHaJ7KyD3g,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0
-0xEqfbgJFmbXh53qSqEww,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0.0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0,0,0,0,0.0,0,0,0,0,0,0,0.0,0,0,0,0.0,0.0,0


In [26]:
X = rating_crosstab.values.T
X.shape

(1600, 13951)

In [27]:
import sklearn
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import accuracy_score


SVD = TruncatedSVD(n_components=12, random_state=17)
result_matrix = SVD.fit_transform(X)
result_matrix.shape

(1600, 12)

In [28]:
corr_matrix = np.corrcoef(result_matrix)
corr_matrix

array([[ 1.        ,  0.29347819,  0.09093931, ..., -0.17646211,
         0.73500005,  0.47118215],
       [ 0.29347819,  1.        , -0.10274908, ...,  0.35035375,
         0.3444479 ,  0.60917881],
       [ 0.09093931, -0.10274908,  1.        , ..., -0.07592776,
         0.26178526, -0.2826825 ],
       ...,
       [-0.17646211,  0.35035375, -0.07592776, ...,  1.        ,
        -0.12478479, -0.07641054],
       [ 0.73500005,  0.3444479 ,  0.26178526, ..., -0.12478479,
         1.        ,  0.62558886],
       [ 0.47118215,  0.60917881, -0.2826825 , ..., -0.07641054,
         0.62558886,  1.        ]])

In [29]:
restaurant_names = rating_crosstab.columns
restaurants_list = list(restaurant_names)


In [30]:
if 'U-Haul Moving & Storage at Southgate' in restaurant_names:
  print(1)

In [31]:
restaurant_names

Index(['#1 Nails', '1 Man 2 Hands', '217', '217 & Smith's Towing',
       '217 Immediate Care', '2M Cleaning Services', '301 Mongolia',
       '309 Green', '4U Nails & Spa', '5 Star Nutrition Champaign',
       ...
       'Yellowfin Japanese Restaurant', 'Yoga Institute of Champaign-Urbana',
       'Yong's Alterations', 'Yori Q Korean Grill & Bar', 'Yuko Hair Salon',
       'Za's Italian Cafe', 'Zelma's', 'Zmodo Technology',
       'Zorba's Restaurant', 'Zyggyz'],
      dtype='object', name='name', length=1600)

In [32]:
res=[]

In [33]:
def compute(row):
  name= row["name"]
  rest = restaurants_list.index(name)

  corr_popular_rest = corr_matrix[rest]

  #print(corr_popular_rest)
  
  try : 
    rest_rate= avg_rating.loc[ restaurant_names[np.argmax(corr_popular_rest)]]
    #print(rest_rate)
    #max_cor= np.max(corr_popular_rest)
    #rest_rate=rest_rate*2
    res.append(rest_rate)
    #res_dict[name]= rest_rate

    
  except KeyError :
    res.append(3)
    

In [34]:
X_test.apply(compute, axis=1)

ValueError: ignored

In [ ]:
len(res)

In [ ]:
yhat= res

In [ ]:
from sklearn.metrics import mean_squared_error
d=mean_squared_error(y_test,yhat)

In [ ]:
d

In [ ]:
from sklearn.metrics import r2_score
r2_score(y_test,yhat)

In [ ]:
len(yhat)

In [ ]:
len(y_test)